In [1]:
!dd if=/kaggle/input/web-scrapyscraper/web_framework_scraper-main/store.db of=/tmp/store.db bs=2M status=progress

17305698304 bytes (17 GB, 16 GiB) copied, 146 s, 119 MB/s
8254+1 records in
8254+1 records out
17311916032 bytes (17 GB, 16 GiB) copied, 146.073 s, 119 MB/s


In [2]:
import sqlite3
con = sqlite3.connect("/tmp/store.db", isolation_level=None)
print(*con.execute("SELECT * FROM progress WHERE data != 'site no longer online' LIMIT 2").fetchall(), sep='\n')
con.close()

('www.filmweb.pl', '{"cloudCheck":{"ipAddresses":["193.200.227.13"],"detectedCloudPlatforms":[]},"whatrunsData":{"date":null,"data":{}}}', 1000, 0)
('znanija.com', '{"cloudCheck":{"ipAddresses":["104.16.209.56","104.16.208.56"],"detectedCloudPlatforms":["Cloudflare"]},"whatrunsData":{"date":1563425101888,"data":{"Widgets":[{"name":"Survicate","category":5,"icon":"Survicate.png","index":"1609","sourceUrl":"https://znanija.com/task/15049934","detectedTime":1563425101888,"latestDetectedTime":1563425101888,"website":"https://www.whatruns.com/click/?target=survicate.com","siteListUrl":"https://www.whatruns.com/technology/survicate"}],"Analytics":[{"name":"Google Analytics","category":10,"icon":"GA.png","index":"274","version":"UA","sourceUrl":"https://znanija.com/task/15049934","detectedTime":1563425101888,"latestDetectedTime":1563425101888,"website":"https://www.whatruns.com/click/?target=google.com%2Fanalytics","siteListUrl":"https://www.whatruns.com/technology/google-analytics"}],"Javasc

In [3]:
import sqlite3
con = sqlite3.connect("/tmp/store.db", isolation_level=None)
con.execute("DROP TABLE IF EXISTS websiteFrameworkData");
con.execute("CREATE TABLE IF NOT EXISTS websiteFrameworkData (idx INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, domain TEXT NOT NULL, rank INTEGER NOT NULL, siteIsOnline INTEGER, numIpAddressesDetected INTEGER, detectedCloudPlatforms TEXT, whatrunsDateUnixMillis TEXT, whatrunsData TEXT)")
con.execute("""
INSERT INTO websiteFrameworkData (domain, rank, siteIsOnline, numIpAddressesDetected, detectedCloudPlatforms, whatrunsDateUnixMillis, whatrunsData) 
SELECT 
    progress.domain as domain, 
    progress.rank as rank,
    data IS NOT NULL AND data != 'site no longer online' as siteIsOnline,
    CASE WHEN data IS NOT NULL AND data != 'site no longer online'
        THEN CAST(json_array_length(json_extract(data, '$.cloudCheck.ipAddresses')) as INTEGER)
        ELSE NULL
    END as numIpAddressesDetected,
    CASE WHEN data IS NOT NULL AND data != 'site no longer online'
        THEN json_extract(data, '$.cloudCheck.detectedCloudPlatforms')
        ELSE NULL
    END as detectedCloudPlatforms,
    CASE WHEN data IS NOT NULL AND data != 'site no longer online'
        THEN CAST(json_extract(data, '$.whatrunsData.date') as INTEGER)
        ELSE NULL
    END as whatrunsDateUnixMillis,
    CASE WHEN data IS NOT NULL AND data != 'site no longer online'
        THEN json_extract(data, '$.whatrunsData.data')
        ELSE NULL
    END as whatrunsData
FROM progress WHERE progress.rank <= 10000000
""")
con.execute("DROP TABLE IF EXISTS progress")
con.execute("VACUUM")
con.close()

In [4]:
#!cp --reflink=always /kaggle/input/web-scrapyscraper/web_framework_scraper-main/store.db /kaggle/working/store.db
!dd if=/tmp/store.db of=/kaggle/working/store.db bs=2M status=progress

15225012224 bytes (15 GB, 14 GiB) copied, 82 s, 186 MB/s
7259+1 records in
7259+1 records out
15225012224 bytes (15 GB, 14 GiB) copied, 82.0046 s, 186 MB/s


In [5]:
import sqlite3
con = sqlite3.connect("/tmp/store.db", isolation_level=None)
print(*con.execute("SELECT * FROM websiteFrameworkData WHERE numIpAddressesDetected > 1 LIMIT 10").fetchall(), sep='\n')
con.close()

(2, 'znanija.com', 1000, 1, 2, '["Cloudflare"]', '1563425101888', '{"Widgets":[{"name":"Survicate","category":5,"icon":"Survicate.png","index":"1609","sourceUrl":"https://znanija.com/task/15049934","detectedTime":1563425101888,"latestDetectedTime":1563425101888,"website":"https://www.whatruns.com/click/?target=survicate.com","siteListUrl":"https://www.whatruns.com/technology/survicate"}],"Analytics":[{"name":"Google Analytics","category":10,"icon":"GA.png","index":"274","version":"UA","sourceUrl":"https://znanija.com/task/15049934","detectedTime":1563425101888,"latestDetectedTime":1563425101888,"website":"https://www.whatruns.com/click/?target=google.com%2Fanalytics","siteListUrl":"https://www.whatruns.com/technology/google-analytics"}],"Javascript Frameworks":[{"name":"React","category":12,"icon":"React.png","index":"611","sourceUrl":"https://znanija.com/task/15049934","detectedTime":1563425101888,"latestDetectedTime":1563425101888,"website":"https://www.whatruns.com/click/?target=fac